In [ ]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [58]:
!pip install torch torchquantum

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
!pip install pennylane pennylane-qiskit qiskit matplotlib

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached mthree-1.0.0-cp37-cp37m-manylinux

In [ ]:
!python supernet/train_search_bank.py --warmup_epochs 0 --steps 13725 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 4 --n_encode_layers 2 --save "16_params_mb_2" --data "./experiment_data/bank" --save_dir "./supernet/bank"

2022-08-16 11:01:34.597135: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-08-16 11:01:34.599130: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-08-16 11:01:34.683961: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(1098, 2, 4) (122, 2, 4)
subnet: [265530, 215937, 329697, 53932], expert_idx: 0
2022-08-16 11:01:49.321966: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-16 11:01:49.331784: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 24999

## Train random circuits

In [ ]:
x_train = np.genfromtxt('./experiment_data/mnist_2/x_train.txt')[:, :16]
x_test = np.genfromtxt('./experiment_data/mnist_2/x_test.txt')[:, :16]

y_train = np.genfromtxt('./experiment_data/mnist_2/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/mnist_2/y_test.txt')
# y_test = 1 - 2 * y_test

num_params = [8, 12, 16, 20]

num_qubits = 4
num_embeds = 16

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random/mnist_2/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(5, 25):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0], 'exp', 'adjoint') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 8000, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list)   

Step 1 | Sliding Loss Window : 4.0
Step 51 | Sliding Loss Window : 2.187600255627598
Step 101 | Sliding Loss Window : 2.2590229983956838
Step 151 | Sliding Loss Window : 1.5320324912141792
Step 201 | Sliding Loss Window : 1.9313105269880635
Step 251 | Sliding Loss Window : 1.7666775834278239
Step 301 | Sliding Loss Window : 2.0750406697118065
Step 351 | Sliding Loss Window : 2.1084439710223695
Step 401 | Sliding Loss Window : 1.9723802921524447
Step 451 | Sliding Loss Window : 2.2454848535795673
Step 501 | Sliding Loss Window : 1.7446556161634108
Step 551 | Sliding Loss Window : 2.2217199699763936
Step 601 | Sliding Loss Window : 2.620143087540319
Step 651 | Sliding Loss Window : 1.8327828452421813
Step 701 | Sliding Loss Window : 1.9538236705186254
Step 751 | Sliding Loss Window : 1.3231223505430676
Step 801 | Sliding Loss Window : 2.2582615921649225
Step 851 | Sliding Loss Window : 1.9677460515391891
Step 901 | Sliding Loss Window : 2.2869731595218945
Step 951 | Sliding Loss Window :

## Train human design circuits

In [ ]:
from datasets import load_dataset
from create_human_design_circs import generate_human_design_circ

dataset = 'bank'
num_reps = 1

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

ours_dir = './human_design/iqp_basic/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train.reshape((len(x_train), 1, x_train.shape[1]))
x_test = x_test.reshape((len(x_test), 1, x_test.shape[1]))

num_params = [4 * i for i in range(1, 6)]
params_per_layer = 4

num_embed_layers = 1

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RY], [qml.RX]] * 4
enc_layer_options = [[1, None]]

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // params_per_layer
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'iqp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options,
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, 5490, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 1.55661032515806
Step 51 | Sliding Loss Window : 0.7734079128016569
Step 101 | Sliding Loss Window : 0.7292371786102528
Step 151 | Sliding Loss Window : 0.900080000146541
Step 201 | Sliding Loss Window : 0.8445123629453514
Step 251 | Sliding Loss Window : 1.0070322165363155
Step 301 | Sliding Loss Window : 0.854661908387086
Step 351 | Sliding Loss Window : 1.118777117268481
Step 401 | Sliding Loss Window : 1.1283580365458363
Step 451 | Sliding Loss Window : 0.9391300397634487
Step 501 | Sliding Loss Window : 0.9874516898262581
Step 551 | Sliding Loss Window : 1.0603554734019156
Step 601 | Sliding Loss Window : 1.1766590878983332
Step 651 | Sliding Loss Window : 1.1450357506622746
Step 701 | Sliding Loss Window : 1.0669217318054267
Step 751 | Sliding Loss Window : 0.9572742640648484
Step 801 | Sliding Loss Window : 1.2453976887305531
Step 851 | Sliding Loss Window : 1.0912665370867907
Step 901 | Sliding Loss Window : 1.2295931635172206
Step 951 | Sliding L

## Train supernet selected ciurcuits

In [119]:
num_searched_params = 12

results_dir = './supernet/fmnist_4/search-{}_params'.format(num_searched_params)

sc_circs = np.genfromtxt(results_dir + '/nas_result_sorted.txt', dtype=str)

circ_layers = [[int(j) for j in i[0].split('-')] for i in sc_circs]
circ_accs = [float(i[1]) for i in sc_circs]

layer_gates = [i[1:].split(')') for i in open(results_dir + '/search_space.txt').read().split('\n')][:-1]

layer_rots = [''.join([j for j in i[0] if j.isupper()]) for i in layer_gates]
layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

layer_cnots = [''.join([j for j in i[1][1:] if j not in ['[', ']', ',', ' ']]) for i in layer_gates]
layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]

In [94]:
layer_rots[32]

['rx', 'rx', 'rx', 'rz']

In [13]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_batched_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'fmnist_4'
main_dir = './supernet/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

inds = np.random.choice(len(x_test), 400, False)
x_test = x_test[inds]
y_test = y_test[inds]

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 2

device_name = 'ibmq_lima'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [20, 24]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0, 1], 'exp')
        
        val_exp_list = []
        
        for j in range(np.ceil(len(x_test) / 32).astype('int32')):
            curr_batch_params = np.concatenate([[curr_params] for k in range(min(32, len(x_test) - 32 * j))], 0)
            
            val_exp_list.append(circ(x_test[32 * j:min(len(x_test), 32* j + 32)], curr_batch_params))
            
        val_exps = np.concatenate(val_exp_list)
        val_losses = np.sum(np.power(val_exps - y_test, 2), 1)
        val_loss = np.mean(val_losses)
        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0) == 2)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet__dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet__dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet__dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

dataset = 'bank'
main_dir = './supernet/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 2)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 2
meas_qubits = [0]
num_steps = 5490

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [4, 8, 12, 16, 20]
search_nums = [100]
num_trials = [25]

device_name = 'ibmq_lima'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
    circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
    circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/{}/search_{}'.format(device_name, search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
            np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

ibmqfactory.load_account:WARNING:2022-10-03 14:21:05,899: Credentials are already in use. The existing account in the session will be replaced.
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (151488.88818503704 > 2 * 66428.77212153834). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (151488.88818503704 > 2 * 66428.77212153834). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (151488.88818503704 > 2 *

0.680327868852459 0.9746578404160796 [285544]
Step 1 | Sliding Loss Window : 2.9823522631777535
Step 51 | Sliding Loss Window : 1.0452587886327027
Step 101 | Sliding Loss Window : 1.0556045355682635
Step 151 | Sliding Loss Window : 1.0559194646333283
Step 201 | Sliding Loss Window : 1.0054267841618652
Step 251 | Sliding Loss Window : 1.05549142506185
Step 301 | Sliding Loss Window : 1.174862284245272
Step 351 | Sliding Loss Window : 0.9811569586371202
Step 401 | Sliding Loss Window : 1.2283661373124481
Step 451 | Sliding Loss Window : 1.015731277717079
Step 501 | Sliding Loss Window : 1.1877894611597324
Step 551 | Sliding Loss Window : 1.090028263911122
Step 601 | Sliding Loss Window : 1.0775702544582506
Step 651 | Sliding Loss Window : 1.151270530646701
Step 701 | Sliding Loss Window : 1.1359511701109712
Step 751 | Sliding Loss Window : 1.0698677124995724
Step 801 | Sliding Loss Window : 1.2453135425168254
Step 851 | Sliding Loss Window : 1.0609652450359062
Step 901 | Sliding Loss Win

## Our metric

In [ ]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets_nt import load_dataset

# ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

dataset = 'vowel_2'

search_params = [16, 20, 24, 28, 32]

num_qubits = 4
num_embeds = 10
meas_qubits = [0]
num_classes = 2
num_params = 32

curr_dir = './ours/{}/'.format(dataset)

if not os.path.exists(curr_dir):
    os.mkdir(curr_dir)

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)
samples_per_class = len(x_train) // num_classes

# class_0_sel = np.random.choice(samples_per_class, 16, False)
# class_1_sel = np.random.choice(samples_per_class, 16, False) + samples_per_class
# class_2_sel = np.random.choice(samples_per_class, 16, False) + (samples_per_class * 2)
# class_3_sel = np.random.choice(samples_per_class, 16, False) + (samples_per_class * 3)
# sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

class_0_sel = np.random.choice(samples_per_class, 16, False)
class_1_sel = np.random.choice(samples_per_class, 16, False) + samples_per_class
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
num_data = len(sel_data)

np.savetxt('./ours/{}/sel_data.txt'.format(dataset), sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_param_dir = curr_dir + '{}_params'.format(p)
    
    if not os.path.exists(curr_param_dir):
        os.mkdir(curr_param_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(2500):
        curr_circ_dir = curr_param_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((num_params, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((32, 32))
        circ_t_mean_mat = np.zeros((32, 32))

        for j in range(num_params):
            curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / num_params
            circ_t_mean_mat += t_mat / num_params
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_param_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_param_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_param_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_param_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_param_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_param_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_param_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_param_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244

## Our metic + fixed embedding

In [ ]:
search_params = [2, 4, 6, 8, 10, 12, 14, 16]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 2
num_embeds = 0

x_train = np.genfromtxt('./experiment_data/moons_300/x_train.txt')
x_test = np.genfromtxt('./experiment_data/moons_300/x_test.txt')

y_train = np.genfromtxt('./experiment_data/moons_300/y_train.txt')[:, 0]
y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/moons_300/y_test.txt')[:, 0]
y_test = 1 - 2 * y_test

class_0_sel = np.random.choice(240, 16, False)
class_1_sel = np.random.choice(240, 16, False) + 240
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

np.savetxt('./ours_fe/moons/angle_1/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours_fe/moons/angle_1/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    for i in range(250):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        circ_gates = ['ry'] * 2 + circ_gates
        gate_params = [[0], [1]] + gate_params
        inputs_bounds = [k for k in range(2)] + [2 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for k in range(2)] + weights_bounds        
           
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
    
        circ_d2_scores = []
        circ_d2_t_scores = []
    
        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(32)]).reshape((32, p))
            mat = compute_reduced_similarity(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - 32) / 996)
        
            diff_mat = mat - ideal
            diff_2d = np.sqrt(np.sum(np.multiply(diff_mat, diff_mat)))
            
            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sqrt(np.sum(np.multiply(diff_t_mat, diff_t_mat)))
            
            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)

## train circuits for our method

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss
from create_noise_models import noisy_dev_from_backend

search_nums = [100]
param_nums = [4, 8, 12, 16, 20]

dataset = 'mnist_2'
num_reps = 1

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]
mat_size = 1024

device_name = 'ibm_nairobi'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

ours_dir = './ours/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        noise_scores = []
        
        for i in range(2500):
            noise_scores.append(np.genfromtxt(ours_dir + '/{}_params/circ_{}/noise_metric/{}/metric_tvd_score.txt'.format(param_num, i + 1, device_name)))
            
        noise_scores = np.array(noise_scores)
        
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))
        perf_scores = (mat_size - mean_t_mat_scores) / mat_size
        combined_scores = np.multiply(perf_scores, noise_scores)
        
        circ_inds = np.random.permutation(2500)
        
        curr_dir = ours_dir + '/{}_params/search_{}_{}'.format(param_num, search_num, device_name)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = circ_inds[range(j * 100, j * 100 + 100)]
            sel_ind = sel_inds[np.argmax(combined_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', combined_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])
            np.savetxt(curr_trial_dir + '/sel_circuit_score.txt', [combined_scores[sel_ind]])
            
            print(sel_inds, combined_scores[sel_inds])
            print(sel_ind, combined_scores[sel_ind])
            print(noise_scores[sel_ind], perf_scores[sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)
        
            losses_list = []
            accs_list = []
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 8000, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)
#                 acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])             

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          
            np.savetxt(curr_trial_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_trial_dir + '/noisy_val_losses.txt', noisy_losses_list) 

ibmqfactory.load_account:WARNING:2022-09-26 18:03:04,365: Credentials are already in use. The existing account in the session will be replaced.


[2002  273 1512  942  708 2356 1672  947  181 2230 1491 2120 1926 1644
  154 2368 1987  923 1211 1326 2277 1322 2466 1920  890 2324  363 1862
 2416 1940 2065 2092  333  256 1776  886 2259 1336 1976 1230  840  525
 1048 1358  902 2264  245 2294  194 1407 1397  184  324 2292  515  969
  250 1063 1540  314  292 1623  785 1299  631 2146 1582  230  165   94
  305 1481   45 2026  690 1690  974 1478  466 1816 1298 1827 1843  885
 1245  892 1098 2268  968  485 2257 1231  352 2144  787  137 2239 1617
  687  623] [0.54729904 0.52635994 0.75771579 0.61070136 0.43130449 0.66438266
 0.68899752 0.69100023 0.61954633 0.441082   0.68816867 0.58170835
 0.74682318 0.81387547 0.6653899  0.42073756 0.42907526 0.45062318
 0.61657194 0.69688536 0.60475775 0.69797991 0.36794759 0.69866997
 0.68814154 0.44725418 0.71446639 0.64455645 0.45646243 0.52842495
 0.75060104 0.66263011 0.75814132 0.44259262 0.40695572 0.46944006
 0.60981251 0.53263637 0.51978767 0.42900467 0.7798522  0.52621746
 0.44392299 0.62389437

## Compute metri forf correlation circuits

In [ ]:
from importlib import reload

from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets import load_dataset

In [61]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets import load_dataset

# ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
# ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

# ideal = 2 * ideal - 1

dataset = 'vowel_2'

curr_dir = f'./experiment_data/{dataset}/trained_circuits'
num_qubits = 4
meas_qubits = [0]

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

# class_0_sel = np.random.choice(2000, 16, False)
# class_1_sel = np.random.choice(2000, 16, False) + 2000
# class_2_sel = np.random.choice(2000, 16, False) + 4000
# class_3_sel = np.random.choice(2000, 16, False) + 6000
# sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

class_0_sel = np.random.choice(len(x_test) // 2, 16, False)
class_1_sel = np.random.choice(len(x_test) // 2, 16, False) + (len(x_test) // 2)
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
# sel_data = np.genfromtxt(curr_dir + '/sel_data.txt')

num_params = 32
num_data = len(sel_data)

d2_min_scores = []
d2_mean_scores = []
d2_var_scores = []
    
d2_t_min_scores = []
d2_t_mean_scores = []
d2_t_var_scores = []

mean_mat_scores = []
mean_t_mat_scores = []

np.savetxt(curr_dir + '/sel_data.txt', sel_data)

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
    
    batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                        weights_bounds, meas_qubits, 'matrix') 
    
    params = 2 * np.pi * np.random.sample((num_params, weights_bounds[-1]))
    
    if not os.path.exists(circ_dir + '/fid_mats'):
        os.mkdir(circ_dir + '/fid_mats')
    
    np.savetxt(circ_dir + '/fid_mats/metric_params.txt', params) 
    
    circ_d2_scores = []
    circ_d2_t_scores = []
    circ_mean_mat = np.zeros((num_data, num_data))
    circ_t_mean_mat = np.zeros((num_data, num_data))
    
    for j in range(num_params):
        curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, weights_bounds[-1]))
        mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
        
        t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
        diff_mat = mat - ideal
        diff_2d = np.sum(np.multiply(diff_mat, diff_mat))
            
        diff_t_mat = t_mat - ideal
        diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))
            
        circ_d2_scores.append(diff_2d)
        circ_d2_t_scores.append(diff_2d_t)
            
        circ_mean_mat += mat / num_params
        circ_t_mean_mat += t_mat / num_params

    np.savetxt(circ_dir + '/fid_mats/d2_scores_2.txt', circ_d2_scores)
    np.savetxt(circ_dir + '/fid_mats/d2_t_scores_2.txt', circ_d2_t_scores)
        
    d2_min_scores.append(np.min(circ_d2_scores))
    d2_mean_scores.append(np.mean(circ_d2_scores))
    d2_var_scores.append(np.var(circ_d2_scores))
        
    d2_t_min_scores.append(np.min(circ_d2_t_scores))
    d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
    d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    diff_mean_mat = ideal - circ_mean_mat
    diff_t_mean_mat = ideal - circ_t_mean_mat   
    
    mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
    mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
    
    np.savetxt(circ_dir + '/fid_mats/mean_t_mat.txt', circ_t_mean_mat)
    
    print(i)
        
np.savetxt(curr_dir + '/d2_mean_scores_2.txt', d2_mean_scores)
np.savetxt(curr_dir + '/d2_min_scores_2.txt', d2_min_scores)
np.savetxt(curr_dir + '/d2_var_scores_2.txt', d2_var_scores)
    
np.savetxt(curr_dir + '/d2_t_mean_scores_2.txt', d2_t_mean_scores)
np.savetxt(curr_dir + '/d2_t_min_scores_2.txt', d2_t_min_scores)
np.savetxt(curr_dir + '/d2_t_var_scores_2.txt', d2_t_var_scores)
    
np.savetxt(curr_dir + '/d2_mean_mat_scores_kta.txt', mean_mat_scores)
np.savetxt(curr_dir + '/d2_mean_t_mat_scores_kta.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## noise metric fo correlation circuits

In [49]:
from importlib import reload

import metrics

reload(metrics)

<module 'metrics' from '/root/metrics.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './ours/{}'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

param_nums = [12, 16]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_lima'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for p in param_nums:
    param_dir = curr_dir + '/{}_params'.format(p)
    
    for i in range(2500):
        circ_dir = param_dir + '/circ_{}'.format(i + 1)

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

        noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noise_metric_dir = circ_dir + '/noise_metric'

        if not os.path.exists(noise_metric_dir):
            os.mkdir(noise_metric_dir)

        device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

        if not os.path.exists(device_noise_metric_dir):
            os.mkdir(device_noise_metric_dir)

        params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
        batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]

        noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
        noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

        actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))

        tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)

        np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
        np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])

        score_tvds.append(1 - tvd)
        actual_tvds.append(actual_tvd)

        print(1 - tvd, actual_tvd, i)

ibmqfactory.load_account:WARNING:2022-09-23 04:01:34,470: Credentials are already in use. The existing account in the session will be replaced.


0.835662841796875 0.8326492309570312 0
0.895050048828125 0.8936004638671875 1
0.8674163818359375 0.8673019409179688 2
0.9376220703125 0.935546875 3
0.9295501708984375 0.925750732421875 4
0.900665283203125 0.8977203369140625 5
0.9154739379882812 0.9134674072265625 6
0.929840087890625 0.923126220703125 7
0.9314041137695312 0.9250640869140625 8
0.8875732421875 0.8960113525390625 9
0.8616409301757812 0.868408203125 10
0.9289932250976562 0.9296798706054688 11
0.6792755126953125 0.6708145141601562 12
0.9002609252929688 0.9106674194335938 13
0.9210739135742188 0.9237289428710938 14
0.912200927734375 0.9166259765625 15
0.8344497680664062 0.8240737915039062 16
0.885009765625 0.8836517333984375 17
0.9276885986328125 0.925994873046875 18
0.8379898071289062 0.8320159912109375 19
0.8497543334960938 0.8773269653320312 20
0.8712615966796875 0.8705368041992188 21
0.9080657958984375 0.915985107421875 22
0.81121826171875 0.7242507934570312 23
0.9179611206054688 0.9151382446289062 24
0.8249664306640625 0

## train correlation circuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibmq_belem'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(600, 800):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (116978.03068342054 > 2 * 52062.3693340039). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (92038.51932346408 > 2 * 1552.3716846118762). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (116978.03068342054 > 2 * 52062.3693340039). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_ae

0.4814943281870684 0.5761336822509766 601


In [56]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'mnist_4'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 16

num_train_steps = 2500
num_test_data = 2000

loss = TQMseLoss(2, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=False)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(200, 400):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, 2, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

ModuleNotFoundError: No module named 'torchquantum'

In [16]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1024, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=100, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(133, 166):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 0.099609375
Step 1001 | Accuracy: 0.2998046875
Step 2001 | Accuracy: 0.2841796875
Step 3001 | Accuracy: 0.310546875
Step 4001 | Accuracy: 0.27734375
Step 5001 | Accuracy: 0.35546875
Step 6001 | Accuracy: 0.3388671875
Step 7001 | Accuracy: 0.3505859375
Step 8001 | Accuracy: 0.310546875
Step 9001 | Accuracy: 0.3369140625
Step 10001 | Accuracy: 0.3544921875
Step 11001 | Accuracy: 0.3095703125
Step 12001 | Accuracy: 0.333984375
Step 13001 | Accuracy: 0.33984375
Step 14001 | Accuracy: 0.3740234375
Step 15001 | Accuracy: 0.3173828125
Step 16001 | Accuracy: 0.2861328125
Step 17001 | Accuracy: 0.3369140625
Step 18001 | Accuracy: 0.357421875
Step 19001 | Accuracy: 0.3466796875
Step 20001 | Accuracy: 0.345703125
Step 21001 | Accuracy: 0.3466796875
Step 22001 | Accuracy: 0.3076171875
Step 23001 | Accuracy: 0.3505859375
Step 24001 | Accuracy: 0.353515625
Step 25001 | Accuracy: 0.3583984375
Step 26001 | Accuracy: 0.3330078125
Step 27001 | Accuracy: 0.3720703125
Step 28001 | Accur

## Run QuantumNAS stuff

Cloning into 'torchquantum'...
remote: Enumerating objects: 11161, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 11161 (delta 23), reused 31 (delta 13), pack-reused 11110
Receiving objects: 100% (11161/11161), 6.22 MiB | 11.24 MiB/s, done.
Resolving deltas: 100% (6180/6180), done.
Checking out files: 100% (154/154), done.
